In [ ]:
# ------------------------------------------
# 統計並列出所.gpx
# ------------------------------------------

from pathlib import Path
import pandas as pd

# ------------------ 參數 ------------------
TARGET_FOLDERS = {"HikingBook_gpx", "HikingNote_gpx"}
OUT_FILENAME   = "file_list.csv"          # 匯出檔名
ENCODING       = "utf-8-sig"              # Excel 友善的 UTF-8 BOM
# -----------------------------------------

def find_project_root(start: Path, targets: set[str]) -> Path:
    """
    從 start 往父層遞迴，找出同時含有所有 targets 的資料夾並回傳路徑；
    若找不到則拋出 FileNotFoundError。
    """
    for cand in [start, *start.parents]:
        if all((cand / t).exists() for t in targets):
            return cand
    raise FileNotFoundError(
        f"從 {start} 往上層都找不到同時含有 {targets} 的資料夾，"
        "請確認目錄結構是否正確。"
    )

# 1️⃣ 自動定位專案根目錄
cwd  = Path.cwd()
ROOT = find_project_root(cwd, TARGET_FOLDERS)
print(f"★ 專案根目錄：{ROOT}")

# 2️⃣ 收集檔案清單
records = []
for subdir in TARGET_FOLDERS:
    folder = ROOT / subdir
    for p in folder.iterdir():            # 不遞迴；需要遞迴改 rglob("*")
        if p.is_file():
            records.append({"folder": subdir, "filename": p.name})

if not records:
    raise RuntimeError("目標資料夾沒有掃到任何檔案，請確認路徑與檔案存在。")

# 3️⃣ 寫出 CSV 到「目前工作目錄」（cwd），方便同一 Notebook 直接讀
df = pd.DataFrame(records)
out_path = cwd / OUT_FILENAME
df.to_csv(out_path, index=False, encoding=ENCODING)

print(f"✅ 完成！共 {len(df)} 筆，已輸出 {out_path}")


★ 專案根目錄：d:\_iSpan\期末專題\analyze_gpx\FileClean
✅ 完成！共 1108 筆，已輸出 d:\_iSpan\期末專題\analyze_gpx\FileClean\file_list.csv


In [ ]:
# ------------------------------------------
# 工人智慧，觀察檔名決定要 保留 & 剔除 的關鍵字
# ------------------------------------------

import pandas as pd, re, pathlib

df = pd.read_csv("file_list.csv")

def normalize(name):
    stem = pathlib.Path(name).stem          # 去掉副檔名
    return re.sub(r"[^A-Za-z\u4e00-\u9fff]", "", stem)  # 只留中英文字

df["normalized"] = df["filename"].map(normalize)
df = df[df["normalized"] != ""]             # 移除被清成空字串者
result = (df.groupby(["folder", "normalized"])
            .size()
            .reset_index(name="count")
            .sort_values("count", ascending=False))

# 輸出成 CSV，方便後續分析
result.to_csv("normalized_counts.csv", index=False, encoding="utf-8-sig")
print("已輸出 normalized_counts.csv")


已輸出 normalized_counts.csv


In [ ]:
# ------------------------------------------
# 保留 536 筆 .gpx
# 剔除 572 筆 .gpx
# 將資料夾重新命名 "Clean_HikingBook_gpx", "Clean_HikingNote_gpx"
# ------------------------------------------

from pathlib import Path
import csv  # 確保已匯入 csv

# 根目錄為當前工作目錄（ANALYZE_GPX）
ROOT = Path.cwd()

# 原始要處理的資料夾
folders = ["HikingBook_gpx", "HikingNote_gpx"]

# 欲保留的關鍵字
pos_keywords = [
    "桃山",
    "桃山單攻",
    "單攻桃山",
    "桃山一日單攻",
    "一日桃山",
    "桃山步道",
]

# 欲剔除的關鍵字
neg_keywords = [
    "二", "兩", "三", "四", "未",
    "喀拉業", "喀拉頁", "喀拉葉", "喀拉曄",
    "克拉葉", "克拉業", "碦拉業", "客拉業",
    "池有", "山屋", "瀑布", "吊橋",
    "去程", "回程", "O型", "o型",
    "品田", "新達", "詩崙", "日本",
    "營地", "撤退", "下", "機車",
    "車", "神木", "休", "4秀", "2天",
]

# 用來儲存每個檔案的處理結果
records: list[tuple[str, str, str]] = []  # (folder, filename, status)

# 處理各資料夾內的 .gpx 檔
for folder in folders:
    dir_path = ROOT / folder
    if not dir_path.exists():
        print(f"資料夾不存在：{dir_path}")
        continue

    for gpx_file in dir_path.glob("*.gpx"):
        name = gpx_file.stem
        has_pos = any(kw in name for kw in pos_keywords)
        has_neg = any(kw in name for kw in neg_keywords)

        if not has_pos or has_neg:
            print(f"刪除：{gpx_file.name}")
            gpx_file.unlink()
            records.append((folder, gpx_file.name, "deleted"))
        else:
            print(f"保留：{gpx_file.name}")
            records.append((folder, gpx_file.name, "kept"))

# 分割 kept / deleted 清單
kept_records = [(f, fn) for f, fn, st in records if st == "kept"]
deleted_records = [(f, fn) for f, fn, st in records if st == "deleted"]

# 輸出兩個 CSV：kept 與 deleted
kept_csv = ROOT / "gpx_kept_list.csv"
deleted_csv = ROOT / "gpx_deleted_list.csv"

with open(kept_csv, "w", newline="", encoding="utf-8-sig") as f:
    writer = csv.writer(f)
    writer.writerow(["folder", "filename"])
    writer.writerows(kept_records)

with open(deleted_csv, "w", newline="", encoding="utf-8-sig") as f:
    writer = csv.writer(f)
    writer.writerow(["folder", "filename"])
    writer.writerows(deleted_records)

print(f"\n已將保留清單輸出到：{kept_csv}")
print(f"已將刪除清單輸出到：{deleted_csv}")

# 最後，將處理過的資料夾重新命名
for folder in folders:
    src = ROOT / folder
    dst = ROOT / f"Clean_{folder}"
    if src.exists():
        src.rename(dst)
        print(f"已將資料夾 '{folder}' 改名為 '{dst.name}'")
    else:
        print(f"找不到資料夾：{src}")


刪除：01-06-2025 一日單攻桃山喀拉葉很滑很難走.gpx
刪除：0228_二訪武陵桃山@2019.gpx
刪除：0301_再訪桃山回程@2019.gpx
保留：0409桃山沒桃子.gpx
保留：0410桃山.gpx
保留：0521單攻桃山.gpx
保留：0829桃山.gpx
保留：1-18桃山.gpx
保留：10-15 桃山單攻.gpx
刪除：10-17桃山喀拉業一日單攻.gpx
保留：1091011-桃山.gpx
保留：1091226桃山單攻.gpx
保留：111.09.25桃山單攻.gpx
保留：111.8.22桃山單攻.gpx
保留：1110115桃山⛰️.gpx
刪除：1110116桃山瀑布步道.gpx
保留：112-9-24桃山單攻.gpx
刪除：112.09.24桃山+喀拉業單登.gpx
刪除：112.11.12 桃山瀑布.gpx
刪除：113.9.21桃山瀑布ok!.gpx
保留：12.03桃山.gpx
刪除：15-09-2024一日單攻武陵四秀少一秀.gpx
保留：20123-9-23爬桃山爬到厭世.gpx
保留：2014-5-25單攻桃山.gpx
刪除：20190204武陵農場桃山瀑布健行之旅.gpx
保留：20200418 桃山.gpx
刪除：20200722桃山喀拉葉1日-修正.gpx
保留：202011 桃山單攻.gpx
保留：20201212 桃山.gpx
保留：2021-11-25 武陵桃山上行.gpx
刪除：2021-11-26 桃山武陵下行.gpx
刪除：2021-11-27 桃山瀑布.gpx
保留：2021-11-28 桃山.gpx
刪除：2021-11-7 桃山-詩崙山-喀拉業山.gpx
保留：2021-11-7 桃山.gpx
保留：2021-12-11 桃山.gpx
刪除：2021-12-18 桃山瀑布.gpx
保留：2021-12-20 桃山單攻.gpx
刪除：2021-9-24 桃山瀑布單攻.gpx
保留：2021-9-26 桃山單攻.gpx
保留：20210320_桃山單攻.gpx
刪除：20210330 武陵四秀-桃山單攻.gpx
保留：20210927桃山單攻.gpx
刪除：20211010 桃山.喀拉業山.gpx
刪除：20211010一日單攻武陵四秀(池有、品田、桃山、喀拉業).gpx
保留：2021101